In [56]:
import gensim
import os
import collections
import smart_open
import random
import pandas as pd

In [3]:
ls -lh ../paraphrases/

total 8,3M
-rw-r--r-- 1 fogside fogside 4,8M ноя 28  2015 corpus.xml
-rw-r--r-- 1 fogside fogside 1,2K апр 16  2015 LICENSE
-rw-r--r-- 1 fogside fogside 3,6M ноя 28  2015 paraphrases.xml


# Paraphrases:
Paraphrase classes: -1: non-paraphrases, 0: loose paraphrases, 1: strict paraphrases

In [60]:
import xml.etree.ElementTree as ET
data=[]
parser = ET.XMLParser(encoding="utf-8")
tree = ET.parse('../paraphrases/paraphrases.xml', parser=parser)
for elem in tree.iter(tag='value'):
    if elem.attrib['name'] == 'id':
        data.append({'id':elem.text})
    if elem.attrib['name'] == 'text_1':
        data[-1]['text_1'] = elem.text
        
    if elem.attrib['name'] == 'text_2':
        data[-1]['text_2'] = elem.text
        
    if elem.attrib['name'] == 'class':
        data[-1]['class'] = elem.text
        
    if elem.attrib['name'] == 'jaccard':
        data[-1]['jaccard'] = elem.text

In [61]:
len(data)

7227

In [65]:
df = pd.DataFrame(data=data)
df.to_csv("../paraphrases/paraphrases.csv", index=False)
df.head()

,class,id,jaccard,text_1,text_2
0,0,1,0.65,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...
1,0,2,0.5,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...
2,0,3,0.611429,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...
3,-1,4,0.324037,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...
4,0,5,0.606218,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...


In [84]:
df.columns

Index(['class', 'id', 'jaccard', 'text_1', 'text_2'], dtype='object')

In [89]:
# from pymystem3 import Mystem
# import re

# mystem = Mystem()

def lemmatize(sent,i):
    return gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(sent), [i])
#      [w.lower() for w in mystem.lemmatize(sent) if re.match('[\w\-]+$', w)]

## Make train corpus

In [90]:
train_corpus = []

for i, item in df.iterrows():
    if (int(item['class']) >= 0):
        train_corpus.append(lemmatize(item.text_1, i))
        train_corpus.append(lemmatize(item.text_2, i))

len(train_corpus)

9290

In [91]:
train_corpus[0]

TaggedDocument(words=['полицейским', 'разрешат', 'стрелять', 'на', 'поражение', 'по', 'гражданам', 'травматикой'], tags=[0])

# Doc2Vec

In [94]:
model = gensim.models.doc2vec.Doc2Vec(size=50, min_count=2, iter=65)

In [97]:
model.build_vocab(train_corpus)

In [99]:
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.iter)

CPU times: user 14.2 s, sys: 1.23 s, total: 15.4 s
Wall time: 10.2 s


4121174

In [100]:
model.infer_vector(['квантовая', 'любовь', 'далека', 'от', 'теории', 'поля'])

array([-0.03325392, -0.00699296,  0.03302972,  0.11073004,  0.01813658,
        0.09369271, -0.14816837,  0.14582737,  0.02075567, -0.05061377,
       -0.01120421, -0.03038704, -0.0412546 , -0.1710465 , -0.08368099,
        0.09616959,  0.08881664, -0.01980344, -0.05494741,  0.03454892,
       -0.19061941,  0.19387472,  0.14936315, -0.06025795, -0.1299188 ,
       -0.07769785,  0.11038404, -0.08143172, -0.04324384,  0.00111169,
       -0.14261447, -0.05900441, -0.11277287,  0.06067004, -0.06014482,
        0.11404239,  0.02307926, -0.12878232, -0.00254873,  0.10933878,
        0.005285  , -0.04058602, -0.04783004, -0.08775134, -0.06257831,
        0.00651267,  0.05885142, -0.09280544,  0.04819974,  0.04014904], dtype=float32)

In [101]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
s1 = 'Отвергнуть щедрый дар'
s2 = 'покупать преданность дарами и наградами'
